<a href="https://colab.research.google.com/github/fernandarrios/tcc_covid_recuperacao/blob/main/TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#To Do:
 
 - Gráfico de análise temporal da evolução dos casos;
 - ~Rodar código que acrescenta uma coluna de id para ordenar o gráfico;~
 - Gráfico com contagem de casos por município(?)

# Importando os dados

Dados até o dia 17/09/22

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling.profile_report import ProfileReport


def plot_chart(desired_variable, df, column, title):
    plt.figure(figsize=(12, 8))
    sns.countplot(y=desired_variable,
                  data=df,
                  palette='rocket',
                  order=column.value_counts().index).set_title(title)
    plt.show()


def fixing_data_type(column, first_symbol, second_symbol):
    try:
        column = column.apply(lambda x : str(x).replace(first_symbol, second_symbol))
        column = pd.to_datetime(column, format='%Y%m%d')
    except ValueError:
        column = column.apply(lambda x : str(x).replace(first_symbol, second_symbol))
        column = pd.to_datetime(column, dayfirst=True)
    return column

In [ ]:
covid_sistema_um = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/XLSX_Sistemas_1.txt', sep='\t')
covid_sistema_dois = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/XLSX_Sistemas_2.txt', sep='\t')
covid_sistema_tres = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/XLSX_Sistemas_3.txt', sep='\t')
covid_sistema_quatro = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/xlsx_sistemas.txt', sep='\t')

covid_sistema_um['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_um['DATA_NOTIFICACAO'], '-', '')
covid_sistema_dois['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_dois['DATA_NOTIFICACAO'], '-', '')
covid_sistema_tres['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_tres['DATA_NOTIFICACAO'], '-', '')
covid_sistema_quatro['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_quatro['DATA_NOTIFICACAO'], '/', '-')

covid_df = pd.concat([covid_sistema_um, covid_sistema_dois, covid_sistema_tres,
                      covid_sistema_quatro])

report = ProfileReport(covid_df, title='Primeira investigação do dataset')
report.to_file('TCC_COVID_REPORT_1.html')

# Análise Exploratória

#### Eliminando colunas que não fazem sentido para o nosso problema e nem faz sentido para conhecermos o dataset

In [ ]:
covid_df.drop(columns=['ID', 'CODIGO', 'DATA_EVOLUCAO', 'MICRO', 'MACRO', 
                       'ETNIA', 'MUNICIPIO_RESIDENCIA', 'DATA_1_SINTOMA', 'URS', 
                       'DATA_ATUALIZACAO', 'CLASSIFICACAO_CASO'], inplace=True)

#### Como vimos no profile report, temos valores para idade que não fazem sentido, vamos investigar melhor

 Alguém de quase 400 anos não faz sentido, pode ter ocorrido algum erro de digitação. 
 
 Vamos analisar melhor os outliers

In [ ]:
covid_df['IDADE'].min()
covid_df['IDADE'].max()

In [ ]:
sns.boxplot(covid_df['IDADE'])
plt.show()

Tem 1127 outliers e 1825 valores nulos, no primeiro momento pensei em só pegar os dois primeiros números e considerar o terceiro como erro de digitação e, mas o preenchimento da faixa etária deve ter ocorrido de forma automática de acordo com o input manual da idade. Decidi então estabelecer um valor máximo de idade de 100 anos, e filtrar todos os valores que fogem disso.

In [ ]:
covid_df = covid_df.loc[covid_df['IDADE'] <= 100]

In [ ]:
sns.boxplot(covid_df['IDADE'])
plt.show()

#### Contagem por faixa etária 

In [ ]:
covid_df['FAIXA_ETARIA'].value_counts()
plot_chart('FAIXA_ETARIA', covid_df, covid_df['FAIXA_ETARIA'],
           'Faixa etária dos casos de covid')

Como podemos ver, as pessoas mais afetadas foram as pessoas de 30 a 39 anos

#### Qual o local que mais publicou informação sobre a covid em MG?

In [ ]:
covid_df['ORIGEM_DA_INFORMACAO'].value_counts()
plot_chart('ORIGEM_DA_INFORMACAO', covid_df, covid_df['ORIGEM_DA_INFORMACAO'],
           'Portal de Publicação')

O maior portal responsável por divulgar os números da COVID-19 em MG foi o ESUS.

#### Quantas pessoas apresentaram comorbidade?

In [ ]:
covid_df['COMORBIDADE'].value_counts()
plot_chart('COMORBIDADE', covid_df, covid_df['COMORBIDADE'],
           'Presença ou não de comorbidade')

#### Qual a raça predominante?

In [ ]:
covid_df['RACA'].value_counts()
plot_chart('RACA', covid_df, covid_df['RACA'], 'Raça dos cidadãos')

#### Quantas pessoas foram internadas?

In [ ]:
covid_df['INTERNACAO'].value_counts()
plot_chart('INTERNACAO', covid_df, covid_df['INTERNACAO'], 'Casos de internação')


#### Elas foram para o uti?

In [ ]:
covid_df['UTI'].value_counts()
plot_chart('UTI', covid_df, covid_df['UTI'], 'Contagem de casos de internação na UTI')

#### Quantas sobreviveram? Quantas morreram?

In [ ]:
covid_df['EVOLUCAO'].value_counts()
plot_chart('EVOLUCAO', covid_df, covid_df['EVOLUCAO'], 'Contagem de evolução dos casos')


Como podemos ver, temos muito mais casos de pessoas recuperadas, então teremos que fazer o balanceamento delas. O método escolhido irá ser o oversampling de óbitos.

# Preparação para o modelo

#### Eliminando colunas que não fazem sentido para o ML

In [ ]:
covid_df.drop(columns=['DATA_NOTIFICACAO', 'ORIGEM_DA_INFORMACAO'], inplace=True)

#### Encoding


In [ ]:
evolucao_dict = {'RECUPERADO': 1, 'EM ACOMPANHAMENTO': 2, 'OBITO': 3}
covid_df['EVOLUCAO_ORDINAL'] = covid_df.EVOLUCAO.map(evolucao_dict)

faixa_idade_dict = {
    '<1ANO' : 1,
    '1 A 9 ANOS': 2,
    '10 A 19 ANOS': 3,
    '20 A 29 ANOS': 4,
    '30 A 39 ANOS': 5,
    '40 A 49 ANOS': 6,
    '50 A 59 ANOS': 7,
    '60 A 69 ANOS': 8,
    '70 A 79 ANOS': 9,
    '80 A 89 ANOS': 10,
    '90 OU MAIS': 11
}
covid_df['FAIXA_IDADE_ORDINAL'] = covid_df.FAIXA_ETARIA.map(faixa_idade_dict)

# Pycharm content

In [ ]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, \
    confusion_matrix, classification_report
from collections import Counter
from pandas_profiling.profile_report import ProfileReport
pd.set_option('display.max_columns', None)


def plot_chart(desired_variable, df, column, title):
    plt.figure(figsize=(12, 8))
    sns.countplot(y=desired_variable,
                  data=df,
                  palette='rocket',
                  order=column.value_counts().index).set_title(title)
    plt.show()


def plot_box_plot(column):
    sns.boxplot(column, palette='rocket')
    plt.show()


def fixing_data_type(column, first_symbol, second_symbol):
    try:
        column = column.apply(lambda x : str(x).replace(first_symbol, second_symbol))
        column = pd.to_datetime(column, format='%Y%m%d')
    except ValueError:
        column = column.apply(lambda x : str(x).replace(first_symbol, second_symbol))
        column = pd.to_datetime(column, dayfirst=True)
    return column


def drop_columns(df, columns):
    df.drop(columns=columns, inplace=True)
    return df


def plot_confusion_matrix(cm, target_names,
                          normalize=False,
                          title='Confusion Matrix') :
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap='rocket')
    plt.title(title)
    plt.colorbar()

    if target_names is not None :
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize :
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])) :
        if normalize :
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else :
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="black" if cm[i, j] > thresh else "white")

    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()

covid_sistema_um = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/data/XLSX_Sistemas_1.txt', sep='\t')
covid_sistema_dois = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/data/XLSX_Sistemas_2.txt', sep='\t')
covid_sistema_tres = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/data/XLSX_Sistemas_3.txt', sep='\t')
covid_sistema_quatro = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/data/xlsx_sistemas.txt', sep='\t')

covid_sistema_um['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_um['DATA_NOTIFICACAO'], '-', '')
covid_sistema_dois['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_dois['DATA_NOTIFICACAO'], '-', '')
covid_sistema_tres['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_tres['DATA_NOTIFICACAO'], '-', '')
covid_sistema_quatro['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_quatro['DATA_NOTIFICACAO'], '/', '-')

covid_df = pd.concat([covid_sistema_um, covid_sistema_dois, covid_sistema_tres,
                      covid_sistema_quatro])

covid_df.info()

# covid_df.reset_index(drop=True, inplace=True)
# report = ProfileReport(covid_df, title='Primeira investigação do dataset')
# report.to_file('TCC_COVID_REPORT_1.html')

covid_df.isnull().sum()

drop_columns(covid_df, ['ID', 'CODIGO', 'DATA_EVOLUCAO', 'ETNIA', 'MUNICIPIO_RESIDENCIA',
                        'MICRO', 'DATA_1_SINTOMA', 'DATA_ATUALIZACAO',
                        'CLASSIFICACAO_CASO'])

year = covid_df['DATA_NOTIFICACAO'].dt.year
year.value_counts()
covid_df = covid_df.loc[covid_df['DATA_NOTIFICACAO'].dt.year != 1957]

fig, ax = plt.subplots()
plt.figure(figsize=(8, 8))
ax.hist(covid_df['DATA_NOTIFICACAO'], bins=50, color='#301934')
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
ax.xaxis.set_minor_locator(mdates.MonthLocator())
ax.set_xlabel('Data dos casos', fontsize=9)
ax.set_ylabel('Ocorrências', fontsize=9)
fig.savefig('Quantidade de casos ao decorrer do tempo.jpeg')
plt.show()

covid_df = covid_df.loc[covid_df['SEXO'] != 'NAO INFORMADO']

# covid_df['IDADE'].min()
# covid_df['IDADE'].max()
# plot_box_plot(covid_df['IDADE'])

covid_df = covid_df.loc[covid_df['IDADE'] <= 100]
# plot_box_plot(covid_df['IDADE'])

# covid_df['FAIXA_ETARIA'].value_counts()
# plot_chart('FAIXA_ETARIA', covid_df, covid_df['FAIXA_ETARIA'],
#            'Faixa etária dos casos de covid')

# covid_df['ORIGEM_DA_INFORMACAO'].value_counts()
# plot_chart('ORIGEM_DA_INFORMACAO', covid_df, covid_df['ORIGEM_DA_INFORMACAO'],
#            'Portal de Publicação')

# covid_df['COMORBIDADE'].value_counts()
# plot_chart('COMORBIDADE', covid_df, covid_df['COMORBIDADE'],
#            'Presença ou não de comorbidade')
#
# covid_df['RACA'].value_counts()
# plot_chart('RACA', covid_df, covid_df['RACA'], 'Raça dos cidadãos')

# covid_df['INTERNACAO'].value_counts()
# plot_chart('INTERNACAO', covid_df, covid_df['INTERNACAO'], 'Casos de internação')
#
# covid_df['UTI'].value_counts()
# plot_chart('UTI', covid_df, covid_df['UTI'], 'Contagem de casos de internação na UTI')
#
# covid_df['EVOLUCAO'].value_counts()
# plot_chart('EVOLUCAO', covid_df, covid_df['EVOLUCAO'], 'Contagem de evolução dos casos')

# Preparação dos dados para o modelo
drop_columns(covid_df, ['DATA_NOTIFICACAO', 'ORIGEM_DA_INFORMACAO', 'IDADE'])

encoded_df = pd.get_dummies(covid_df, columns=['SEXO', 'INTERNACAO', 'UTI', 'RACA', 'MACRO',
                                               'COMORBIDADE', 'URS'], drop_first=True,
                            dtype='int64')

faixa_idade_dict = {
    '<1ANO' : 1,
    '1 A 9 ANOS': 2,
    '10 A 19 ANOS': 3,
    '20 A 29 ANOS': 4,
    '30 A 39 ANOS': 5,
    '40 A 49 ANOS': 6,
    '50 A 59 ANOS': 7,
    '60 A 69 ANOS': 8,
    '70 A 79 ANOS': 9,
    '80 A 89 ANOS': 10,
    '90 OU MAIS': 11
}
evolucao_dict = {'RECUPERADO': 1, 'EM ACOMPANHAMENTO': 2, 'OBITO': 3}

encoded_df['FAIXA_IDADE_ORDINAL'] = encoded_df.FAIXA_ETARIA.map(faixa_idade_dict)
encoded_df['EVOLUCAO_ORDINAL'] = encoded_df.EVOLUCAO.map(evolucao_dict)
drop_columns(encoded_df, ['FAIXA_ETARIA', 'EVOLUCAO'])

# encoded_df[['FAIXA_IDADE_ORDINAL', 'EVOLUCAO_ORDINAL']] = encoded_df[['FAIXA_IDADE_ORDINAL', 'EVOLUCAO_ORDINAL']] .astype('str')

# corr = encoded_df.corr()
# plt.figure(figsize=(13, 13))
# cmap = sns.diverging_palette(230, 20, as_cmap =True)
# sns.heatmap(corr, cmap=cmap, center=0, vmin=-1, vmax=1, annot=True, square=True,
#             linewidths=5, cbar_kws={'shrink': 0.5})
# # plt.show()
# plt.tight_layout()
# plt.savefig('Correlation.jpeg')

y = encoded_df['EVOLUCAO_ORDINAL']
X = drop_columns(encoded_df, ['EVOLUCAO_ORDINAL'])

sampling_strategy_under = {1: 624100, 2: 116135, 3: 108466}
under = RandomUnderSampler(replacement=True, sampling_strategy=sampling_strategy_under,
                           random_state=42)
# print('Original dataset shape %s' % Counter(y))
X_resampled, y_resampled = under.fit_resample(X, y)
# print(f'Resampled dataset shape whith RandonUnderSample: {Counter(y_resampled)}.')

sampling_strategy_over = {1: 624100, 2: 620999, 3: 620999}
over = RandomOverSampler(sampling_strategy=sampling_strategy_over, random_state=42)
X_combined_sampling, y_combined_sampling = over.fit_resample(X_resampled, y_resampled)
# print(f'Combined RandomOverSampling with RandonUnderSampling: '
#       f'{Counter(y_combined_sampling)}')

X_train, X_test, y_train, y_test = train_test_split(X_combined_sampling, y_combined_sampling,
                                                    test_size=0.2, random_state=42)

# Decision Tree
model = DecisionTreeClassifier(criterion='entropy')
model.fit(X_train, y_train)
y_pred_tree = model.predict(X_test)

accuracy_score(y_true=y_test, y_pred=y_pred_tree)
recall_score(y_true=y_test, y_pred=y_pred_tree, average='micro')


cnf_matrix = confusion_matrix(y_test, y_pred_tree)

plot_confusion_matrix(cnf_matrix,
                      target_names=['Recuperado', 'Em acompanhamento', 'Óbito'])
plt.show()
plt.savefig('Confusion_matrix.jpeg')

# accuracy_score(y_test, y_pred_tree)
# precision_score(y_true=y_test, y_pred=y_pred_tree, average='micro')
# recall_score(y_true=y_test, y_pred=y_pred_tree, average='micro')
report = classification_report(y_true=y_test, y_pred=y_pred_tree)

In [ ]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, \
    confusion_matrix, classification_report
from collections import Counter
from pandas_profiling.profile_report import ProfileReport
pd.set_option('display.max_columns', None)


def fixing_data_type(column, first_symbol, second_symbol):
    try:
        column = column.apply(lambda x : str(x).replace(first_symbol, second_symbol))
        column = pd.to_datetime(column, format='%Y%m%d')
    except ValueError:
        column = column.apply(lambda x : str(x).replace(first_symbol, second_symbol))
        column = pd.to_datetime(column, dayfirst=True)
    return column


def drop_columns(df, columns):
    df.drop(columns=columns, inplace=True)
    return df


def plot_chart(desired_variable, df, column, title, xlabel, ylabel, figsize=(12, 8)):
    plt.figure(figsize=figsize)
    sns.countplot(y=desired_variable,
                  data=df,
                  palette='rocket',
                  order=column.value_counts().index).set_title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(axis='x', linestyle='--')
    plt.minorticks_on()


def plot_box_plot(column, title, xlabel):
    sns.boxplot(column, palette='rocket').set_title(title)
    plt.xlabel(xlabel)
    plt.grid(axis='x', linestyle='--')
    plt.minorticks_on()


def plot_time_series(column, xlabel, ylabel):
    fig, ax = plt.subplots()
    plt.figure(figsize=(8, 8))
    ax.hist(column, bins=50, color='#301934')
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.set_xlabel(xlabel, fontsize=9)
    ax.set_ylabel(ylabel, fontsize=9)


def save_fig_show(figname):
    plt.savefig(f'{figname}.jpeg')
    return plt.show()


def plot_confusion_matrix(cm, target_names,
                          normalize=False,
                          title='Confusion Matrix') :
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap='rocket')
    plt.title(title)
    plt.colorbar()

    if target_names is not None :
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize :
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])) :
        if normalize :
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else :
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="black" if cm[i, j] > thresh else "white")

    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()


covid_sistema_um = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/data/XLSX_Sistemas_1.txt', sep='\t')
covid_sistema_dois = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/data/XLSX_Sistemas_2.txt', sep='\t')
covid_sistema_tres = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/data/XLSX_Sistemas_3.txt', sep='\t')
covid_sistema_quatro = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/data/xlsx_sistemas.txt', sep='\t')

covid_sistema_um['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_um['DATA_NOTIFICACAO'], '-', '')
covid_sistema_dois['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_dois['DATA_NOTIFICACAO'], '-', '')
covid_sistema_tres['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_tres['DATA_NOTIFICACAO'], '-', '')
covid_sistema_quatro['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_quatro['DATA_NOTIFICACAO'], '/', '-')

covid_df = pd.concat([covid_sistema_um, covid_sistema_dois, covid_sistema_tres,
                      covid_sistema_quatro])

print(covid_df.info())

covid_df.reset_index(drop=True, inplace=True)
report = ProfileReport(covid_df, title='Primeira investigação do dataset')
report.to_file('TCC_COVID_REPORT_1.html')

# TRATAMENTO DOS DADOS
print(covid_df.isnull().sum())

drop_columns(covid_df, ['ID', 'CODIGO', 'DATA_EVOLUCAO', 'ETNIA', 'MUNICIPIO_RESIDENCIA',
                        'MICRO', 'DATA_1_SINTOMA', 'DATA_ATUALIZACAO',
                        'CLASSIFICACAO_CASO'])

print(f"Searching for the outlier: \n{covid_df['DATA_NOTIFICACAO'].dt.year.value_counts()}")
print(covid_df.loc[covid_df['DATA_NOTIFICACAO'].dt.year == 1957])
covid_df = covid_df.loc[covid_df['DATA_NOTIFICACAO'].dt.year != 1957]
print(f"After deleting the outlier:\n{covid_df['DATA_NOTIFICACAO'].dt.year.value_counts()}")

plot_time_series(covid_df['DATA_NOTIFICACAO'], 'Date', 'Number of cases')
save_fig_show('Number of cases over time.')

covid_df = covid_df.loc[covid_df['SEXO'] != 'NAO INFORMADO']

plot_box_plot(covid_df['IDADE'], 'Age of patients', 'Age')
save_fig_show('Age')

print(f"Looking for the outliers: " 
      f"{covid_df.loc[covid_df['IDADE'] > 100, ['IDADE', 'FAIXA_ETARIA']]}")

covid_df = covid_df.loc[covid_df['IDADE'] <= 100]
plot_box_plot(covid_df['IDADE'], 'Age of patients', 'Age')
save_fig_show('Age_filtered')

print(covid_df.duplicated().sum())

covid_df.fillna(method='ffill', inplace=True)

print(covid_df.duplicated().sum())


plot_chart(covid_df['MACRO'], covid_df, covid_df['MACRO'], 'Cases by Macroregion',
           'Cases', 'Macroregion', figsize=(16, 8))
save_fig_show('Macroregion_cases')

plot_chart(covid_df['URS'], covid_df, covid_df['URS'], 'Cases by URS', 'Cases', 'URS',
           figsize=(18, 10))
save_fig_show('Cases_by_urs')


# plot_chart('FAIXA_ETARIA', covid_df, covid_df['FAIXA_ETARIA'],
#            'Age Range of COVID-19 cases', 'Case Count', 'Age Range')
# plt.figure(figsize=(15, 10))
# covid_df['FAIXA_ETARIA'].value_counts().sort_index().plot.bar()
# plt.show()
# # ANÁLISE EXPLORATÓRIA
# # covid_df['FAIXA_ETARIA'].value_counts()
# plot_chart('FAIXA_ETARIA', covid_df, covid_df['FAIXA_IDADE_ORDINAL'],
#            'Age Range of COVID-19 cases', 'Case Count', 'Age Range')
# save_fig_show('Age_range_count')

# covid_df['ORIGEM_DA_INFORMACAO'].value_counts()
# plot_chart('ORIGEM_DA_INFORMACAO', covid_df, covid_df['ORIGEM_DA_INFORMACAO'],
#            'Portal de Publicação')

# covid_df['COMORBIDADE'].value_counts()
# plot_chart('COMORBIDADE', covid_df, covid_df['COMORBIDADE'],
#            'Presença ou não de comorbidade')
#
# covid_df['RACA'].value_counts()
# plot_chart('RACA', covid_df, covid_df['RACA'], 'Raça dos cidadãos')

# covid_df['INTERNACAO'].value_counts()
# plot_chart('INTERNACAO', covid_df, covid_df['INTERNACAO'], 'Casos de internação')
#
# covid_df['UTI'].value_counts()
# plot_chart('UTI', covid_df, covid_df['UTI'], 'Contagem de casos de internação na UTI')
#

print(covid_df['EVOLUCAO'].value_counts())
plot_chart('EVOLUCAO', covid_df, covid_df['EVOLUCAO'], 'Case evolution count',
           'Case count', 'Case evolution', figsize=(17, 8))
save_fig_show('Case_evolution_unbalanced')

# Preparação dos dados para o modelo
drop_columns(covid_df, ['DATA_NOTIFICACAO', 'ORIGEM_DA_INFORMACAO', 'IDADE'])

encoded_df = pd.get_dummies(covid_df, columns=['SEXO', 'INTERNACAO', 'UTI', 'RACA', 'MACRO',
                                               'COMORBIDADE', 'URS'], drop_first=True,
                            dtype='int64')

faixa_idade_dict = {
    '<1ANO' : 1,
    '1 A 9 ANOS': 2,
    '10 A 19 ANOS': 3,
    '20 A 29 ANOS': 4,
    '30 A 39 ANOS': 5,
    '40 A 49 ANOS': 6,
    '50 A 59 ANOS': 7,
    '60 A 69 ANOS': 8,
    '70 A 79 ANOS': 9,
    '80 A 89 ANOS': 10,
    '90 OU MAIS': 11
}
evolucao_dict = {'RECUPERADO': 1, 'EM ACOMPANHAMENTO': 2, 'OBITO': 3}

encoded_df['FAIXA_IDADE_ORDINAL'] = encoded_df.FAIXA_ETARIA.map(faixa_idade_dict)
encoded_df['EVOLUCAO_ORDINAL'] = encoded_df.EVOLUCAO.map(evolucao_dict)
drop_columns(encoded_df, ['FAIXA_ETARIA', 'EVOLUCAO'])

encoded_df.isnull().sum()
# corr = encoded_df.corr()
# plt.figure(figsize=(13, 13))
# cmap = sns.diverging_palette(230, 20, as_cmap =True)
# sns.heatmap(corr, cmap=cmap, center=0, vmin=-1, vmax=1, annot=True, square=True,
#             linewidths=5, cbar_kws={'shrink': 0.5})
# # plt.show()
# plt.tight_layout()
# plt.savefig('Correlation.jpeg')

y = encoded_df['EVOLUCAO_ORDINAL']
X = drop_columns(encoded_df, ['EVOLUCAO_ORDINAL'])

sampling_strategy_under = {1: 624100, 2: 116135, 3: 108466}
under = RandomUnderSampler(replacement=True, sampling_strategy=sampling_strategy_under,
                           random_state=42)
print('Original dataset shape %s' % Counter(y))
X_resampled, y_resampled = under.fit_resample(X, y)
print(f'Resampled dataset shape whith RandonUnderSample: {Counter(y_resampled)}.')

sampling_strategy_over = {1: 624100, 2: 620999, 3: 620999}
over = RandomOverSampler(sampling_strategy=sampling_strategy_over, random_state=42)
X_combined_sampling, y_combined_sampling = over.fit_resample(X_resampled, y_resampled)
print(f'Combined RandomOverSampling with RandonUnderSampling: '
      f'{Counter(y_combined_sampling)}')

X_train, X_test, y_train, y_test = train_test_split(X_combined_sampling, y_combined_sampling,
                                                    test_size=0.4, random_state=42)
print(f'X test: {X_test.shape}. \nX train: {X_train.shape}. \ny test: {y_test.shape}.\n'
      f'y train: {y_train.shape}')
# Decision Tree
model = DecisionTreeClassifier(criterion='entropy')
model.fit(X_train, y_train)
y_pred_tree = model.predict(X_test)

accuracy_score(y_true=y_test, y_pred=y_pred_tree)
recall_score(y_true=y_test, y_pred=y_pred_tree, average='micro')


cnf_matrix = confusion_matrix(y_test, y_pred_tree)

plot_confusion_matrix(cnf_matrix,
                      target_names=['Recuperado', 'Em acompanhamento', 'Óbito'])
plt.show()
plt.savefig('Confusion_matrix.jpeg')

# accuracy_score(y_test, y_pred_tree)
# precision_score(y_true=y_test, y_pred=y_pred_tree, average='micro')
# recall_score(y_true=y_test, y_pred=y_pred_tree, average='micro')
report = classification_report(y_true=y_test, y_pred=y_pred_tree)